# ALFA-BD - Conversão de Áudio para Texto

## Leitura de texto dos Alunos do 2ª e 3º Anos do Fundamental

### Referencias:

- David Amos. **The Ultimate Guide To Speech Recognition With Python** - 2018. Disponível em https://realpython.com/python-speech-recognition/


#### Autor: TS#03

Esta notebook tem como objetivo converter os áudios de leitura de alunos para texto com o intuito de aplicar tecnicas de Processamento de Linguagem Natural bem como ánalise exploratória e a implementação como prova de conceito de um modelo preditivo para avaliação da fluência em leitura.

In [18]:
import glob

# Noise reduce
from scipy.io import wavfile
import noisereduce as nr
import soundfile as sf

# Convert
import pandas as pd
import speech_recognition as sr

In [19]:
files = glob.glob('dataset/*.wav')
files

['dataset/9_3_publica_menina_regular.wav',
 'dataset/3_2_publica_menina_bom.wav',
 'dataset/11_2_publica_menino_regular.wav',
 'dataset/12_3_publica_menino_regular.wav',
 'dataset/6_2_publica_menino_ruim.wav',
 'dataset/5_2_publica_menina_regular.wav',
 'dataset/7_3_particular_menina_bom.wav',
 'dataset/4_2_publica_menino_ruim.wav',
 'dataset/8_3_particular_menino_regular.wav',
 'dataset/2_2_particular_menino_regular.wav',
 'dataset/13_3_publica_menino_regular.wav',
 'dataset/10_3_publica_menina_regular.wav',
 'dataset/1_2_particular_menina_bom.wav']

### Remoção de Ruidos - Implementação feita pelo TS#01

In [7]:
def alfa_bd_AT(src):
    for file in src:
        data, rate = sf.read(file)
        noise_clip = data / 1.15
        noise_reduced = nr.reduce_noise(audio_clip=data, noise_clip=noise_clip, prop_decrease=1.0,  verbose=False)
        wavfile.write(file.replace('dataset', 'treated'), rate, noise_reduced)

In [8]:
alfa_bd_AT(files)

### Conversão para Texto

Obs: Os áudios foram comprimidos para redução de tamanho e tempo de processamento:
link https://www.compresss.com/compress-audio.html

Optamos por não utilizar os áudios tratrados pela função acima, pois o Google não obteve um resultado satisfatório na conversão para texto.

O espectro de potência de cada fragmento do áudio, que é essencialmente um gráfico da potência do sinal em função da frequência, é mapeado para um vetor numérico conhecido como coeficientes cepstrais utilizado para reconhecimento de áudio para texto. 

Acreditamos que o tratamento de ruidos do áudio precisa ser refinado para que ele não tire propriedades importantes utilizadas pelo recognize_google.

In [20]:
files = glob.glob('dataset/*.wav')
files

['dataset/9_3_publica_menina_regular.wav',
 'dataset/3_2_publica_menina_bom.wav',
 'dataset/11_2_publica_menino_regular.wav',
 'dataset/12_3_publica_menino_regular.wav',
 'dataset/6_2_publica_menino_ruim.wav',
 'dataset/5_2_publica_menina_regular.wav',
 'dataset/7_3_particular_menina_bom.wav',
 'dataset/4_2_publica_menino_ruim.wav',
 'dataset/8_3_particular_menino_regular.wav',
 'dataset/2_2_particular_menino_regular.wav',
 'dataset/13_3_publica_menino_regular.wav',
 'dataset/10_3_publica_menina_regular.wav',
 'dataset/1_2_particular_menina_bom.wav']

In [21]:
data = [['id', 'ano', 'instituicao', 'genero', 'avaliacao', 'texto']]

recognizer = sr.Recognizer()
recognizer.energy_threshold = 50
recognizer.dynamic_energy_threshold = False

In [22]:
for file in files:
    with sr.AudioFile(file) as source:
        audio = recognizer.record(source)
        
        # remove the effect of noise
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
        text = recognizer.recognize_google(audio, language="pt-BR", show_all=True)
        if len(text) > 0:
            file = file.replace('dataset/', '').replace('.wav', '')
            row = file.split('_')
            row.append(text['alternative'][0]['transcript'])
            data.append(row)
        else:
            print (f'Not converted: {file}')

Not converted: dataset/6_2_publica_menino_ruim.wav
Not converted: dataset/1_2_particular_menina_bom.wav


Obs: Os aúdios não convertidos pelo recognize_google foram convertidos manualmente

In [23]:
# save converted texts
df = pd.DataFrame(data=data)
df.to_csv("dataset/speech_to_text.csv", header=False, index=False)